In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = 10019052 # 店舗ID
SPREADSHEET_KEY = '13sblW2DW2bCa7_V3JksNUwk5sfkH0yTVci58C8oYW4o' # スプレッドシートのID
pstart = 1 #1 パチンコの1台目
sstart = 481 # スロットの1台目
wait = 0.02

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "434":
                    p4table.append(li)
                if unit == "108":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "434":
                        p4table.append(li)
                    if unit == "108":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass
    
# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "543":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "543":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass
    
wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/19', '025606', '1', '9744.0', '904.0', '6', '1']
['08/19', '025606', '2', '2821.0', '-2406.0', '0', '0']
['08/19', '025606', '3', '10769.0', '-7823.0', '2', '2']
['08/19', '025606', '4', '2051.0', '-1805.0', '0', '0']
['08/19', '025606', '5', '1026.0', '-1203.0', '0', '0']
['08/19', '025606', '6', '2308.0', '2107.0', '3', '1']
['08/19', '025606', '7', '15385.0', '3913.0', '12', '3']
['08/19', '025606', '9', '9744.0', '-2406.0', '4', '2']
['08/19', '025665', '10', '36410.0', '-7221.0', '19', '7']
['08/19', '025665', '11', '27692.0', '5719.0', '21', '5']
['08/19', '025665', '12', '38205.0', '-15045.0', '13', '4']
['08/19', '025665', '13', '27692.0', '2107.0', '18', '5']
['08/19', '025665', '14', '33333.0', '-11735.0', '13', '4']
['08/19', '025665', '15', '22821.0', '12038.0', '20', '2']
['08/19', '025665', '16', '43333.0', '13543.0', '33', '5']
['08/19', '025665', '17', '34103.0', '-11735.0', '12', '5']
['08/19', '025665', '18', '40769.0', '-4212.0', '21', '5']
['08/19', '025665', '

['08/19', '025895', '146', '35385.0', '-17453.0', '11', '4']
['08/19', '025895', '147', '25385.0', '-19559.0', '3', '3']
['08/19', '025895', '148', '39231.0', '-11133.0', '17', '4']
['08/19', '025895', '149', '41282.0', '603.0', '26', '5']
['08/19', '025895', '150', '38462.0', '-26781.0', '9', '6']
['08/19', '025895', '151', '21282.0', '-8124.0', '9', '4']
['08/19', '025895', '152', '36667.0', '-29490.0', '3', '1']
['08/19', '025895', '153', '30256.0', '27686.0', '41', '8']
['08/19', '025895', '154', '31282.0', '26783.0', '40', '9']
['08/19', '025895', '155', '32821.0', '-9930.0', '13', '1']
['08/19', '025895', '156', '32821.0', '-1805.0', '24', '9']
['08/19', '025895', '157', '41538.0', '8126.0', '34', '9']
['08/19', '025895', '158', '30256.0', '-7522.0', '14', '4']
['08/19', '025895', '159', '36667.0', '5117.0', '29', '9']
['08/19', '025895', '160', '38205.0', '-5717.0', '20', '3']
['08/19', '025825', '161', '5897.0', '-5115.0', '0', '0']
['08/19', '025825', '162', '6410.0', '-5416.0

['08/19', '025879', '299', '10256.0', '-6319.0', '3', '2']
['08/19', '025878', '300', '8718.0', '-7522.0', '0', '0']
['08/19', '025892', '301', '4103.0', '-902.0', '8', '3']
['08/19', '025865', '302', '3846.0', '3311.0', '20', '3']
['08/19', '025891', '303', '26410.0', '-10832.0', '26', '18']
['08/19', '025862', '304', '21795.0', '-8425.0', '19', '10']
['08/19', '025575', '305', '7949.0', '-1203.0', '11', '4']
['08/19', '025844', '306', '14872.0', '-1805.0', '13', '4']
['08/19', '025647', '307', '1795.0', '4214.0', '11', '1']
['08/19', '025891', '308', '31282.0', '-1203.0', '41', '15']
['08/19', '025693', '309', '0', '0', '1', '1']
['08/19', '025774', '312', '3077.0', '-1203.0', '6', '1']
['08/19', '025523', '313', '2308.0', '1205.0', '6', '1']
['08/19', '025846', '314', '5897.0', '-1504.0', '14', '4']
['08/19', '025847', '315', '10513.0', '-4212.0', '20', '5']
['08/19', '025846', '316', '1026.0', '-902.0', '3', '2']
['08/19', '025722', '317', '11282.0', '904.0', '14', '6']
['08/19', '

['08/19', '025838', '441', '36154.0', '-19559.0', '10', '3']
['08/19', '025838', '442', '20256.0', '18658.0', '27', '3']
['08/19', '025661', '443', '9487.0', '18056.0', '16', '1']
['08/19', '025687', '444', '27692.0', '9931.0', '29', '6']
['08/19', '025687', '445', '20256.0', '-15647.0', '2', '2']
['08/19', '025870', '446', '24872.0', '18959.0', '27', '6']
['08/19', '025850', '447', '16410.0', '-15045.0', '0', '0']
['08/19', '025868', '448', '24615.0', '-3309.0', '26', '10']
['08/19', '025696', '449', '19231.0', '-3610.0', '15', '4']
['08/19', '025764', '450', '24615.0', '9330.0', '19', '5']
['08/19', '025764', '451', '16923.0', '30394.0', '28', '2']
['08/19', '025764', '452', '30513.0', '-12638.0', '8', '2']
['08/19', '025849', '453', '20256.0', '14144.0', '21', '4']
['08/19', '025849', '454', '18718.0', '-7221.0', '6', '2']
['08/19', '025856', '455', '37949.0', '10232.0', '29', '4']
['08/19', '025856', '456', '40513.0', '-11735.0', '16', '5']
['08/19', '025856', '457', '25641.0', '-1

['08/19', '120122', '575', '4591', '-2824.0', '28', '13', '--']
['08/19', '120122', '576', '4732', '-2593.0', '29', '8', '--']
['08/19', '120122', '577', '6183', '-1528.0', '55', '14', '--']
['08/19', '120010', '578', '8385', '556.0', '35', '22', '--']
['08/19', '120010', '579', '2820', '-1111.0', '7', '6', '--']
['08/19', '120010', '580', '5994', '-1250.0', '19', '16', '--']
['08/19', '120010', '581', '5901', '-972.0', '21', '11', '--']
['08/19', '120010', '582', '7647', '556.0', '32', '23', '--']
['08/19', '120010', '583', '3493', '-370.0', '11', '10', '--']
['08/19', '120010', '584', '5042', '-833.0', '14', '14', '--']
['08/19', '120010', '585', '6651', '231.0', '27', '15', '--']
['08/19', '120010', '586', '2542', '-324.0', '9', '3', '--']
['08/19', '120010', '587', '4882', '-1111.0', '15', '14', '--']
['08/19', '120010', '588', '6743', '926.0', '29', '18', '--']
['08/19', '120010', '589', '6403', '648.0', '29', '13', '--']
['08/19', '120010', '590', '6444', '-139.0', '26', '14', '-

['08/19', '120138', '709', '2372', '-1065.0', '0', '3', '4']
['08/19', '120142', '710', '6241', '-231.0', '0', '13', '60']
['08/19', '120142', '711', '4458', '-1250.0', '54', '16', '--']
['08/19', '120142', '712', '4793', '-1852.0', '53', '12', '--']
['08/19', '120053', '713', '5849', '1065.0', '0', '15', '20']
['08/19', '120053', '714', '3293', '417.0', '0', '9', '11']
['08/19', '120053', '715', '5025', '602.0', '0', '10', '11']
['08/19', '120094', '716', '3047', '694.0', '0', '3', '12']
['08/19', '120094', '717', '1344', '-324.0', '0', '1', '5']
['08/19', '119940', '718', '2435', '1111.0', '9', '0', '7']
['08/19', '119940', '719', '1807', '509.0', '5', '0', '4']
['08/19', '120121', '720', '2645', '3472.0', '0', '2', '61']
['08/19', '120121', '721', '1952', '-1944.0', '0', '3', '10']
['08/19', '120121', '722', '1734', '-1620.0', '0', '1', '9']
['08/19', '120121', '723', '2052', '2870.0', '0', '3', '37']
['08/19', '120121', '724', '2046', '1944.0', '0', '3', '38']
['08/19', '120121', '

{'spreadsheetId': '13sblW2DW2bCa7_V3JksNUwk5sfkH0yTVci58C8oYW4o',
 'updates': {'spreadsheetId': '13sblW2DW2bCa7_V3JksNUwk5sfkH0yTVci58C8oYW4o',
  'updatedRange': 'S5RAW!F6872:M6910',
  'updatedRows': 39,
  'updatedColumns': 8,
  'updatedCells': 312}}